<a href="https://colab.research.google.com/github/OGhub/AIFFEL_quest_cr/blob/master/Exploration/Ex07/newssummarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54387 sha256=c8c6e1c182cfc7193cc79c991a3926784b24ab3ec1b881d0a7b43de1f03b133c
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [3]:
from importlib.metadata import version
import nltk
import tensorflow
import summa
import pandas as pd

print("nltk 버전:", nltk.__version__)
print("TensorFlow 버전:", tensorflow.__version__)
print("Pandas 버전:", pd.__version__)
print("Summa 버전:", version('summa'))

nltk 버전: 3.9.1
TensorFlow 버전: 2.17.1
Pandas 버전: 2.2.2
Summa 버전: 1.2.0


In [4]:
import urllib.request
import pandas as pd

# 데이터 다운로드
url = "https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv"
filename = "news_summary_more.csv"
urllib.request.urlretrieve(url, filename)

# 데이터 로드
data = pd.read_csv(filename, encoding='iso-8859-1')

In [5]:
data.sample(10)

,headlines,text
29368,Use 'Saam Dam Dand Bhed' in bypolls: Maha CM i...,Maharashtra CM Devendra Fadnavis allegedly tol...
80047,New study challenges 100-year-old law on fluid...,"Using 3D imaging, UK-based researchers have di..."
97476,Yogi left free to do religious duties: Sena on...,Taking a dig at BJP over appointing two deputy...
77477,"Used to get scared of online rape, acid attack...","Dia Mirza, while talking about being trolled o..."
60634,"Home Minister urges Gates to adopt 1,000 Maois...",Home Minister Rajnath Singh on Thursday urged ...
60631,Hardik Patel slams BJP using 1970s film song,Patidar leader Hardik Patel on Friday shared h...
3898,Twitter still vulnerable: Security firm which ...,"UK-based cybersecurity firm Insinia Security, ..."
7228,"Jethmalani, BJP file joint plea to withdraw ex...",Senior lawyer Ram Jethmalani and BJP have file...
46178,UP sugar firm booked for cheating bank of Ã¢Â...,The CBI has registered a case against Uttar Pr...
98273,"Digvijaya came to Goa for luxury, not work: Pa...","After the Goa Assembly floor test, CM Manohar ..."


In [6]:
import nltk
print("NLTK 버전:", nltk.__version__)

# 필요한 리소스 다운로드
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

NLTK 버전: 3.9.1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
import re
import nltk
from nltk.corpus import stopwords

# NLTK 데이터 다운로드 (필요한 경우)
nltk.download('stopwords')
nltk.download('punkt')

# 수축어 사전 정의 (예시)
contraction_mapping = {
    "can't": "cannot",
    "won't": "will not",
    # 필요한 수축어들을 추가하세요.
}

def preprocess_text(text, remove_stopwords=True):
    # 수축어 치환
    for contraction, expansion in contraction_mapping.items():
        text = text.replace(contraction, expansion)

    # 특수문자 및 숫자 제거
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # 소문자 변환
    text = text.lower()

    # 불용어 제거 (옵션)
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        words = nltk.word_tokenize(text)
        words = [word for word in words if word not in stop_words]
        text = ' '.join(words)
    else:
        text = ' '.join(nltk.word_tokenize(text))

    return text

# 요약문에 시작 및 종료 토큰 추가 (전처리 이전에 한 번만 추가)
data['headlines'] = data['headlines'].apply(lambda x: 'starttoken ' + x + ' endtoken')

# 원문과 요약문에 전처리 적용
# 원문: 불용어 제거 적용
data['text'] = data['text'].apply(lambda x: preprocess_text(x, remove_stopwords=True))

# 요약문: 불용어 제거 미적용
data['headlines'] = data['headlines'].apply(lambda x: preprocess_text(x, remove_stopwords=False))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 파라미터 설정
max_text_len = 80
max_summary_len = 10

# 인코더 입력 데이터 토큰화
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(data['text'])
x_sequences = x_tokenizer.texts_to_sequences(data['text'])
x_padded = pad_sequences(x_sequences, maxlen=max_text_len, padding='post')

# 디코더 입력 데이터 토큰화
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(data['headlines'])
y_sequences = y_tokenizer.texts_to_sequences(data['headlines'])
y_padded = pad_sequences(y_sequences, maxlen=max_summary_len, padding='post')

# 단어 집합 크기
x_vocab_size = len(x_tokenizer.word_index) + 1
y_vocab_size = len(y_tokenizer.word_index) + 1

# 디코더 입력과 출력 데이터 설정
decoder_input_data = y_padded[:, :-1]
decoder_target_data = y_padded[:, 1:]

# 디코더 출력 데이터를 3차원으로 변환
decoder_target_data = decoder_target_data.reshape(decoder_target_data.shape[0], decoder_target_data.shape[1], 1)



In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model

# 인코더 정의
encoder_inputs = Input(shape=(max_text_len,))
enc_emb = Embedding(x_vocab_size, 128, trainable=True)(encoder_inputs)
encoder_lstm = LSTM(256, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# 디코더 정의
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_vocab_size, 128, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# 어텐션 메커니즘
attn_layer = tf.keras.layers.Attention()
attn_out = attn_layer([decoder_outputs, encoder_outputs])

# 어텐션 결과와 디코더 출력 결합
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attn_out])

# 출력층
decoder_dense = Dense(y_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

# 최종 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
# 모델 컴파일 및 학습
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
history = model.fit(
    [x_padded, decoder_input_data],
    decoder_target_data,
    epochs=30,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 212s 152ms/step - loss: 6.8782 - val_loss: 6.4682
Epoch 2/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 264s 154ms/step - loss: 6.4290 - val_loss: 6.4493
Epoch 3/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 262s 154ms/step - loss: 6.3905 - val_loss: 6.4260
Epoch 4/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 265s 156ms/step - loss: 6.3476 - val_loss: 6.4067
Epoch 5/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 259s 154ms/step - loss: 6.3051 - val_loss: 6.3276
Epoch 6/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 262s 154ms/step - loss: 6.1553 - val_loss: 6.2021
Epoch 7/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 262s 154ms/step - loss: 5.9652 - val_loss: 5.9834
Epoch 8/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 262s 154ms/step - loss: 5.7082 - val_loss: 5.7969
Epoch 9/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 264s 155ms/step - loss: 5.4883 - val_loss: 5.6312
Epoch 10/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 260s 154ms/step - loss: 5.2787 - val_loss: 5.5057
Epoch 11/30
1384/1384 ━━━━━━━━━━━━━━━━━━━━ 262s 154ms/step - loss: 5.0804 - val

In [22]:
# 인코더 모델
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

# 디코더 상태 입력
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_hidden_state_input = Input(shape=(max_text_len, 256))

# 디코더 임베딩
dec_emb2 = dec_emb_layer(decoder_inputs)

# 디코더 LSTM
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c]
)

# 어텐션 적용
attn_out_inf = attn_layer([decoder_outputs2, decoder_hidden_state_input])
decoder_inf_concat = Concatenate(axis=-1)([decoder_outputs2, attn_out_inf])

# 출력층
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2]
)


In [23]:
import numpy as np

reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.index_word
target_word_index = y_tokenizer.word_index

def decode_sequence(input_seq):
    # 인코더 예측
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # 디코더 입력 초기화
    target_seq = np.array([[target_word_index['starttoken']]])
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + [e_out, e_h, e_c]
        )

        # 단어 예측
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index.get(sampled_token_index, '')

        if sampled_token == 'endtoken' or len(decoded_sentence.split()) >= (max_summary_len - 1):
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_token

            # 디코더 입력 업데이트
            target_seq = np.array([[sampled_token_index]])
            e_h, e_c = h, c

    return decoded_sentence.strip()

In [24]:
# 결과 비교
for i in range(5):
    print("원문:", data['text'].iloc[i])
    print("실제 요약:", data['headlines'].iloc[i])
    print("예측 요약:", decode_sequence(x_padded[i].reshape(1, max_text_len)))
    print("\n")


원문: saurav kant alumnus upgrad iiit b pg program machine learning artificial intelligence sr systems engineer infosys almost years work experience program upgrad degree career support helped transition data scientist tech mahindra salary hike upgrad online power learning powered lakh careers
실제 요약: starttoken starttoken upgrad learner switches career ml al salary hike endtoken endtoken
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
예측 요약: starttoken microsoft transition transition transition data


원문: kunal shah credit card bill payment platform cred gave users chance win free food swiggy one year pranav kaushik delhi techie bagged reward spending cred coins users get one cred coin per rupee bill paid used avail rewards brands like